In [ ]:
period = '_1'
temp1 = [pl.DataFrame() for _ in range(4)] # creating an empty dataframe holder list obj

In [ ]:
# FOR IID LEVEL Data - > 

In [ ]:
# current_vol , prior_vol, vol_change.
def process_1():

    res = []

    for i in range(0,4):
        #i = 0
        g = levels[i]
        
        source_df = (ln2.select([g,'plan_type','plan_type_group1','plan_type_group2','payer_id','IID','product_id',f'TUF{period}c',f'TUF{period}p']))
        
        df = (
            source_df
            .join(top_hcps[i],on = [g,'plan_type','payer_id','IID'],how = 'inner')
            .drop(['plan_type_group1','plan_type_group2'])
        )
        
        df_t = (
            source_df
            .join(top_hcps[i].filter(plan_type = 'Total'),left_on = [g,'plan_type_group1','payer_id','IID'],right_on = [g,'plan_type','payer_id','IID'],how = 'inner')
            .drop(['plan_type_group2','plan_type'])
            .rename({'plan_type_group1' : 'plan_type'})
            .select(df.columns)
        )
        
        df_pdc = (
            source_df
            .join(top_hcps[i].filter(plan_type = 'Part D and Commercial'),left_on = [g,'plan_type_group2','payer_id','IID'],right_on = [g,'plan_type','payer_id','IID'],how = 'inner')
            .drop(['plan_type_group1','plan_type'])
            .rename({'plan_type_group2' : 'plan_type'})
            .select(df.columns)
        )
        
        df = df.vstack(df_t).vstack(df_pdc)
        
        df = (
            df
            .rename({f'TUF{period}c':'cur_vol',f'TUF{period}p':'pri_vol'})
            .with_columns(
                vol_change = pl.col('cur_vol') - pl.col('pri_vol'),
                prc_vol_growth = (pl.col('cur_vol')/pl.col('pri_vol'))-1,
            )
            .with_columns(
                pl.when(pl.col('vol_change')/pl.col('pri_vol') > 0.02).then(pl.lit('P'))
                .when(pl.col('vol_change')/pl.col('pri_vol') < -0.02).then(pl.lit('Q'))
                .when(pl.col('vol_change')==0).then(None)
                .otherwise(None).alias('vol_change_ind')
            )
        )

        res.append(df)
    return (res)

temp1 = process_1()